In [1]:
import pandas as pd
import numpy as np
from ase import Atoms
from tqdm import tqdm

from Constants import Keys, Dataset
from Dataset import (
    create_df,
    create_tf_dataset,
)
from Training import start_training

In [2]:
df_temp = pd.read_pickle("./../Datasets/df_8molecules.pkl.gzip", compression="gzip")
df_temp

,name,ase_atoms,energy,forces
0,aspirin0,"[(Atom('C', [2.2393, -0.3791, 0.263], index=0)...",[-406757.5912640221],"[[-1.8342235724199345, 1.0243422546435585, -2...."
1,aspirin1,"[(Atom('C', [2.2407929, -0.3792177, 0.26220864...",[-406757.03429293],"[[-1.0413302733349663, -2.6158647255196934, -1..."
2,aspirin2,"[(Atom('C', [2.24227707, -0.37935822, 0.261403...",[-406755.654207047],"[[0.12499747050397911, -6.797555259687784, -0...."
3,aspirin3,"[(Atom('C', [2.2437641, -0.37955814, 0.2605934...",[-406753.83377323527],"[[1.4975764196670198, -11.182962368191696, 0.4..."
4,aspirin4,"[(Atom('C', [2.24526922, -0.37985629, 0.259784...",[-406752.02145761694],"[[2.923970894394034, -15.430847763868533, 1.51..."
...,...,...,...,...
795,uracil95,"[(Atom('C', [1.61046731, 0.33167008, 0.2059238...",[-260104.45884442827],"[[22.168542461178145, -7.322201361941002, -8.9..."
796,uracil96,"[(Atom('C', [1.60611853, 0.33264749, 0.2082256...",[-260105.17779926624],"[[24.394914606755318, -8.58992759527803, -9.79..."
797,uracil97,"[(Atom('C', [1.60193117, 0.3335614, 0.21046942...",[-260105.30795649372],"[[26.220437611461865, -9.57431090684579, -10.5..."
798,uracil98,"[(Atom('C', [1.59796762, 0.3343929, 0.21262356...",[-260104.75759818061],"[[27.532219946059733, -10.318331382787903, -11..."


In [4]:
files = ["./../Datasets/Cu_df2_1k.pkl.gzip", 
         "./../Datasets/ethanol.pckl.gzip"]
create_df(files, grid_size=1, save_path="./../Datasets/Cu-Ethanol_df_preprocessed")

Reading ./../Datasets/Cu_df2_1k.pkl.gzip
Reading ./../Datasets/ethanol.pckl.gzip


,name,ase_atoms,energy_corrected,forces,energy_corrected_per_atom,energy,grid,local_distance_matrix,local_atomic_numbers,long_range_distance_matrix,long_range_atomic_features,n_max_local,n_max_long_range
0,A1:FHI-aims/PBE/tight:phonon:supercell_phonon_0,"(Atom('Cu', [4.3368086899420173e-19, 0.0070709...",-236.789603,"[[-1.13852957740976e-06, -0.0464638907314277, ...",-3.699838,NaN,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...","[[[2.5576886661519227, -1.300522737268895e-16,...","[[[29.0, 29.0], [29.0, 29.0], [29.0, 29.0], [0...","[[[13.579369421666973, -0.5348199739239784, -0...","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,63
1,A1:FHI-aims/PBE/tight:qha:phonopy_strain_1_02:...,"(Atom('Cu', [0.0, 0.007070999999999999, 0.0070...",-236.667372,"[[-1.7950713845026e-06, -0.0426786586259085, -...",-3.697928,NaN,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...","[[[2.5743339597810535, 0.0, -0.707106781186547...","[[[29.0, 29.0], [29.0, 29.0], [29.0, 29.0], [0...","[[[5.158667823661648, 0.0, -0.7071067811865476...","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,63
2,fcc_100surf_X110_Y-110_Z001_2at_d1.05,"(Atom('Cu', [1.2699999999999998, 1.26999999999...",-7.393033,"[[1.43567879745929e-07, 1.29341091704393e-08, ...",-3.696516,NaN,"[GridCell(index: (1, 1, 0), atoms: Atom('Cu', ...","[[[2.5399998414568845, 0.5000000312092766, 0.5...","[[[29, 29], [0, 0], [0, 0], [0, 0], [0, 0], [0...","[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], ...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",1,0
3,A1:FHI-aims/PBE/tight:tp:trigonal:tp_4_12500,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",-7.391491,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]",-3.695746,NaN,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...","[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], ...","[[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0....","[[[6.419884, 0.0, -3.4587011996639086e-17, -1....","[[[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1
4,fcc_110surf_X100_Y01-1_Z011_2at_d1.00,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",-7.391475,"[[-9.87788653663608e-08, -6.83294122864741e-08...",-3.695738,NaN,"[GridCell(index: (0, 0, 0), atoms: Atom('Cu', ...","[[[2.590799824143888, -0.70710673319015, -0.50...","[[[29, 29], [0, 0], [0, 0], [0, 0], [0, 0], [0...","[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], ...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,NaN,"(Atom('C', [-0.32652388, -0.51658872, -0.03294...",-44.429389,"[[2.7201568578213875, 3.66578381198858, 2.4756...",-4.936599,-4209.436457,"[GridCell(index: (1, 0, 0), atoms: Atom('C', [...","[[[1.0343774225255842, 0.35159739770035886, 0....","[[[6, 1], [6, 1], [0, 0], [0, 0], [0, 0], [0, ...","[[[1.696334685076114, 0.3541336065842723, -0.9...","[[[6, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0...",3,6
1834,NaN,"(Atom('C', [-0.33627583, -0.50508262, -0.15660...",-44.491822,"[[-0.9712440517099267, 3.475118895210679, -0.2...",-4.943536,-4209.498890,"[GridCell(index: (1, 0, 0), atoms: Atom('C', [...","[[[1.4250485855668362, -0.9668569647061895, -0...","[[[6, 8], [6, 1], [6, 1], [0, 0], [0, 0], [0, ...","[[[1.6849636601019067, 0.2820904932580285, -0....","[[[6, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0...",4,7
1835,NaN,"(Atom('C', [0.07746533, -0.35826784, 0.4507782...",-44.620031,"[[0.12325057119995139, 0.8819713337546561, -0....",-4.957781,-4209.627098,"[GridCell(index: (1, 0, 1), atoms: Atom('C', [...","[[[1.0462284679619804, 0.326336312244571, 0.94...","[[[6, 1], [6, 6], [6, 1], [6, 8], [6, 1], [6, ...","[[[2.0943709880650925, 0.3568483779898367, -0....","[[[6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",6,5
1836,NaN,"(Atom('C', [0.18038169, -0.30347999, -0.353913...",-44.399892,"[[-0.07392296614804496, 1.7788623335772662, 1....",-4.933321,-4209.406960,"[GridCell(index: (2, 0, 1), atoms: Atom('C', [...","[[[1.020589821866751, 0.21938583474256218, 0.9...","[[[6, 1], [6, 8], [6, 1], [6, 1], [0, 0], [0, ...","[[[1.640426146895227, 0

In [8]:
create_df(["./../Datasets/df_8molecules.pkl.gzip"], grid_size=1, save_path="./../Datasets/df_8molecules_preprocessed")

Reading ./../Datasets/df_8molecules.pkl.gzip


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Atoms'

In [6]:
path = "./../Datasets/Cu-Ethanol_df_preprocessed_grid_size_1.pkl.gzip"
ds = create_tf_dataset(path)

--------------------------------------------------
Dataset shapes:
(20147, 8, 4)
(20147, 8, 2)
(20147, 124, 4)
(20147, 124, 101)
(20147, 3)
--------------------------------------------------


In [2]:
start_training()

--------------------------------------------------
Dataset shapes:
(20147, 8, 4)
(20147, 8, 2)
(20147, 124, 4)
(20147, 124, 101)
(20147, 3)
--------------------------------------------------


2024-01-19 11:03:58,096 - tensorflow - WARNING - From C:\ProgramData\Anaconda3\envs\ml-potential\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_local_matrix (InputL  [(None, 8, 4)]               0         []                            
 ayer)                                                                                            
                                                                                                  
 input_long_range_matrix (I  [(None, 124, 4)]             0         []                            
 nputLayer)                                                                                       
                                                                                                  
 lambda (Lambda)             (None, 8)                    0         ['input_local_matrix[0][0]']  
                                                                                              

2024-01-19 11:03:59,100 - tensorflow - WARNING - From C:\ProgramData\Anaconda3\envs\ml-potential\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.



176/314 [===============>..............] - ETA: 1s - loss: 16325773.0000

KeyboardInterrupt: 

In [11]:
data = np.load("./../Datasets/md17_aspirin.npz")
print(data.files)
data['E'].shape

['E', 'name', 'F', 'theory', 'R', 'z', 'type', 'md5']


(211762, 1)

In [12]:
from Dataset_NP import create_dataset

files = ['./../Datasets/md17_aspirin.npz', 
         './../Datasets/md17_benzene2017.npz',
         './../Datasets/md17_ethanol.npz', 
         './../Datasets/md17_malonaldehyde.npz',
         './../Datasets/md17_naphthalene.npz',
         './../Datasets/md17_salicylic.npz',
         './../Datasets/md17_toluene.npz',
         './../Datasets/md17_uracil.npz']

save_path = "./../Datasets/df_8molecules.npz"
create_dataset(files, grid_size=1, save_path=save_path, n_samples_per=1000)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 21 and the array at index 1 has size 12

In [5]:
# convert to pandas dataframe
def convert_to_df(data, n_samples=1000, offset=0):
    # create df with columns = ['name', 'ase_atoms', 'energy', 'forces']
    df = pd.DataFrame(columns=['name', 'ase_atoms', 'energy', 'forces'])
    
    # loop over all samples
    if n_samples > len(data['R']):
        n_samples = len(data['R'])
    
    for i in tqdm(range(offset, n_samples), desc="Converting file to df"):
        # generate name (convert data['name'] b-string to string)
        name = data['name'].tobytes().decode('utf-8') + str(i)
        
        # create ase atoms object
        ase_atoms = []
        for j in range(len(data['z'])):
            # create ase atoms object
            ase_atoms.append(Atoms(numbers=[data['z'][j]], positions=[data['R'][i][j]]))
        
        df.loc[i] = [name, ase_atoms, data['E'][i], data['F'][i]]
        
    # return random sample of df
    return df

# convert multiple npz files to pandas dataframe
def convert_files_to_df(files, n_samples_per=1000, val_split=0.1):
    # create df with columns = ['name', 'ase_atoms', 'energy', 'forces']
    train_df = pd.DataFrame(columns=['name', 'ase_atoms', 'energy', 'forces'])
    val_df = pd.DataFrame(columns=['name', 'ase_atoms', 'energy', 'forces'])
    
    # loop over all files
    for file in files:
        try:
            # load data
            data = np.load(file)
            train_count = int(n_samples_per * (1 - val_split))
            val_count = n_samples_per - train_count
            # convert to df
            df_temp = convert_to_df(data, n_samples=train_count, offset=0)
            # concatenate to df
            train_df = pd.concat([train_df, df_temp], ignore_index=True)
            df_temp = convert_to_df(data, n_samples=val_count, offset=train_count)
            val_df = pd.concat([val_df, df_temp], ignore_index=True)
            
        except Exception as e:
            print(f"Error '{e}' in file: {file}")
        
    # return random sample of df
    return train_df, val_df

In [7]:
files = ['./../Datasets/md17_aspirin.npz', 
         './../Datasets/md17_benzene2017.npz',
         './../Datasets/md17_ethanol.npz', 
         './../Datasets/md17_malonaldehyde.npz',
         './../Datasets/md17_naphthalene.npz',
         './../Datasets/md17_salicylic.npz',
         './../Datasets/md17_toluene.npz',
         './../Datasets/md17_uracil.npz']

train_df, val_df = convert_files_to_df(files, n_samples_per=550)

# save df
train_df.to_pickle("./../Datasets/train_df_8_molecules.pkl.gzip", compression="gzip")
val_df.to_pickle("./../Datasets/val_df_8_molecules.pkl.gzip", compression="gzip")

Converting file to df:   8%|▊         | 44/550 [07:47<1:29:36, 10.63s/it]


Error Unable to allocate 102. MiB for an array with shape (13341006,) and data type float64 in file: ./../Datasets/md17_aspirin.npz


Converting file to df:   5%|▍         | 26/550 [02:58<59:55,  6.86s/it]  


Error Unable to allocate 172. MiB for an array with shape (22607388,) and data type float64 in file: ./../Datasets/md17_benzene2017.npz


Converting file to df:   7%|▋         | 39/550 [03:24<44:39,  5.24s/it]


...

Converting file to df:   5%|▌         | 29/550 [05:28<1:38:22, 11.33s/it]


Error Unable to allocate 152. MiB for an array with shape (19925550,) and data type float64 in file: ./../Datasets/md17_toluene.npz


Converting file to df:  22%|██▏       | 120/550 [04:33<16:18,  2.28s/it]


Error Unable to allocate 36.7 MiB for an array with shape (4815720,) and data type float64 in file: ./../Datasets/md17_uracil.npz


ValueError: too many values to unpack (expected 2)

In [4]:
df

,name,ase_atoms,energy,forces
0,aspirin0,"[(Atom('C', [2.2393, -0.3791, 0.263], index=0)...",[-406757.5912640221],"[[-1.8342235724199345, 1.0243422546435585, -2...."
1,qmC6H60,"[(Atom('C', [0.0, 1.397, 0.0], index=0)), (Ato...",[-146536.1068],"[[4.7384784e-11, -3.8259648, -6.71719679999999..."
2,ethanol0,"[(Atom('C', [0.0072, -0.5687, 0.0], index=0)),...",[-97208.40600498248],"[[1.4517297437249999, 6.01923561735, 5.2067503..."
3,malonaldehyde0,"[(Atom('C', [1.1705, 0.2287, -0.3792], index=0...",[-167510.9957431333],"[[5.41593353586545, -26.65568358505657, 20.368..."
4,naphthalene0,"[(Atom('C', [-1.2248, -1.3976, -0.0002], index...",[-241923.48033636645],"[[1.771664427555, -3.11529186255, 0.3342713034..."
5,salicylic0,"[(Atom('C', [1.6347, -0.2302, 0.0038], index=0...",[-311033.6045437261],"[[24.100793243792506, 34.751210343679325, 0.62..."
6,C6H5CH30,"[(Atom('C', [2.4329, 0.0, 0.001], index=0)), (...",[-170244.27638720776],"[[-11.377063537039927, 0.008422566012263665, -..."
7,uracil0,"[(Atom('C', [1.632, 0.2951, -0.0637], index=0)...",[-260120.1104989303],"[[0.8587629092226489, 7.9378343937156775, -0.6..."
